In [3]:
import os
import random
import shutil, os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt   
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pylab as pl
from matplotlib.pyplot import figure
from collections import Counter

from os import listdir
from os.path import isfile, join

In [5]:
truth_file = pd.read_csv('/Users/sleung2/Documents/MIDS Program/Capstone_local/snapshot_wisconsin/all/yolo_splits3/test/test_labels.csv')
truth_file = truth_file.rename(columns = {'TRIGGER_ID': 'event_id',
                                               'CLASS_SPECIES': 'true_class',
                                               'Total': 'true_count',
                                               'CLASS_SPECIES_RESTATED': 'true_class_name'})

truth_file_no_other = truth_file[truth_file['true_class_name'] != 'other']

In [6]:
labels = pd.DataFrame(['foxgray_foxred',
              'cottontail_snowshoehare',
              'raccoon',
              'opossum',
              'turkey',
              'bear',
              'elk',
              'deer',
              'coyote',
              'wolf']).sort_values(0)
labels = labels.rename(columns = {0: 'species'})
labels.insert(0, 'class', range(0, len(labels)))

# Phase 1- Blank

## Yolo

In [40]:
#Read lines from txt results file

toppath = 'phase 1-blank/'
txt_file = 'phase1_yolo.txt'

with open(toppath + txt_file, 'r') as f:
    blank_yolo_lines = f.readlines()

In [94]:
yolo_image_prediction_list = []
yolo_blank_prediction_list = []


for line in blank_yolo_lines:
    
    image_name = line.split('Filename: ')[1].split('; ')[0]
    prediction = line.split('; ')[1].split('; Bbox[list]')[0]
    yolo_image_prediction_list.append(image_name)
    yolo_blank_prediction_list.append(prediction)

In [95]:
yolo_stage_1_prediction_df = pd.DataFrame({'image_name': yolo_image_prediction_list, 'prediction': yolo_blank_prediction_list})



In [96]:
yolo_stage_1_prediction_df.sort_values(by = 'image_name')

,image_name,prediction
1743,SSWI000000002741773A.jpg,
3471,SSWI000000002741773B.jpg,
3270,SSWI000000002741773C.jpg,
578,SSWI000000004032002A.jpg,"Class:0.0,Conf:0.8916367292404175;"
3220,SSWI000000004032002B.jpg,"Class:0.0,Conf:0.8409765362739563;"
...,...,...
2245,SSWI000000023514111B.jpg,"Class:0.0,Conf:0.694103479385376;"
2956,SSWI000000023514111C.jpg,"Class:0.0,Conf:0.8248010277748108;"
1527,SSWI000000023514155A.jpg,"Class:0.0,Conf:0.3264309763908386;Class:0.0,Co..."
1003,SSWI000000023514155B.jpg,"Class:0.0,Conf:0.7902511358261108;"


In [97]:
yolo_stage_1_prediction_df['event_id'] = yolo_stage_1_prediction_df['image_name'].apply(lambda x: x[:-5] if 'SSWI' in x else x.split('_')[0])

In [98]:
yolo_stage_1_prediction_df['class_name'] = yolo_stage_1_prediction_df['prediction'].apply(lambda x: 'empty' if x == '' else 'animal')
yolo_stage_1_prediction_df['class'] = yolo_stage_1_prediction_df['class_name'].apply(lambda x: 0 if x == 'animal' else 1)



In [99]:
yolo_stage_1_prediction_df['conf'] = 

,image_name,prediction,event_id,class_name,class
0,SSWI000000019125218C.jpg,,SSWI000000019125218,empty,1
1,SSWI000000020163117C.jpg,"Class:0.0,Conf:0.5518747568130493;",SSWI000000020163117,animal,0
2,SSWI000000016424935C.jpg,,SSWI000000016424935,empty,1
3,SSWI000000017050425B.jpg,"Class:0.0,Conf:0.8673955798149109;",SSWI000000017050425,animal,0
4,SSWI000000019521970B.jpg,"Class:0.0,Conf:0.7018851041793823;",SSWI000000019521970,animal,0
...,...,...,...,...,...
3606,SSWI000000021927444C.jpg,"Class:0.0,Conf:0.6779311299324036;",SSWI000000021927444,animal,0
3607,SSWI000000006772893B.jpg,,SSWI000000006772893,empty,1
3608,SSWI000000013550296B.jpg,"Class:0.0,Conf:0.36881259083747864;Class:0.0,C...",SSWI000000013550296,animal,0
3609,SSWI000000020300636B.jpg,"Class:0.0,Conf:0.27924853563308716;Class:0.0,C...",SSWI000000020300636,animal,0


In [143]:
yolo_stage_1_prediction_grouped = pd.pivot_table(yolo_stage_1_prediction_df, index = 'event_id',
              columns = 'class_name',
              values = 'class',
              aggfunc=len, fill_value=0).reset_index()



In [2]:
top_path = '/Users/sleung2/Documents/MIDS Program/capstone/ensemble/phase 1-blank/'
yolo_file = 'phase1_yolo.txt'
effnet_file = 'phase1_efficientnetb0_classifications.json'



In [4]:
def images_to_events(image_df, image_id_col = 'id'):
    '''Convert images to events'''

    return image_df['id'].apply(lambda x: x[:-5] if 'SSWI' in x else x.split('_')[0])

def group_events(prediction_image_df):
    '''Convert predictions in terms of events'''
    prediction_event_df = pd.pivot_table(prediction_image_df, index = 'event_id',
              columns = 'class_name',
              values = 'class',
              aggfunc=len, fill_value=0).reset_index() #len(x.unique())).fillna(0)
    return prediction_event_df

def interpret_event_predictions(prediction_event_df):
    return prediction_event_df.apply(lambda x : 'empty' if x['animal'] == 0 else 'animal', axis = 1)

## YOLO


with open(top_path + yolo_file, 'r') as f:
    blank_yolo_lines = f.readlines()

yolo_image_prediction_list = []
yolo_blank_prediction_list = []


for line in blank_yolo_lines:

    image_name = line.split('Filename: ')[1].split('.jpg;')[0]
    prediction = line.split('.jpg;')[1].split('; Bbox[list]')[0]
    yolo_image_prediction_list.append(image_name)
    yolo_blank_prediction_list.append(prediction)

yolo_stage_1_prediction_df = pd.DataFrame({'id': yolo_image_prediction_list, 'image_prediction': yolo_blank_prediction_list})

#Create necessary columns
yolo_stage_1_prediction_df['event_id'] = images_to_events(yolo_stage_1_prediction_df)




In [6]:
yolo_stage_1_prediction_df['class_name'] = yolo_stage_1_prediction_df['prediction'].apply(lambda x: 'empty' if x == '' else 'animal')

KeyError: 'prediction'

## Effnet

In [41]:
#Read lines from json results file
    
effnet_stage_1 = pd.read_json(toppath + 'phase1_efficientnetb0_classifications.json')
effnet_stage_1 = effnet_stage_1['phase1_classification_results'].apply(pd.Series)


effnet_stage_1['event_id'] = effnet_stage_1['id'].apply(lambda x: x[:-5] if 'SSWI' in x else x.split('_')[0])


In [43]:
effnet_stage_1['conf'].describe()

count    7280.000000
mean        0.921996
std         0.129911
min         0.500286
25%         0.902891
50%         0.993798
75%         0.999491
max         1.000000
Name: conf, dtype: float64

In [44]:
effnet_stage_1_pivot = pd.pivot_table(effnet_stage_1, index = 'event_id',
              columns = 'class_name',
              values = 'class',
              aggfunc=len, fill_value=0) #len(x.unique())).fillna(0)



In [47]:
effnet_stage_1_pivot['event_prediction'] = effnet_stage_1_pivot.apply(lambda x : 'empty' if x['animal'] == 0 else 'animal', axis = 1)

In [48]:
effnet_stage_1_pivot = effnet_stage_1_pivot.reset_index()

In [68]:
empty_correct_count = effnet_stage_1_pivot[(~(effnet_stage_1_pivot['event_id'].str.contains('SSWI'))& 
                       (effnet_stage_1_pivot['event_prediction'] == 'animal'))]

In [66]:
animal_correct_count = effnet_stage_1_pivot[((effnet_stage_1_pivot['event_id'].str.contains('SSWI'))& 
                       (effnet_stage_1_pivot['event_prediction'] == 'empty'))]

In [69]:
empty_correct_count

class_name,event_id,animal,empty,event_prediction
19,2009543,3,0,animal
20,2009651,2,1,animal
22,2009968,2,1,animal
30,2010190,3,0,animal
47,2011922,3,0,animal
...,...,...,...,...
1205,5031502,3,0,animal
1207,5032116,1,2,animal
1217,5068980,3,0,animal
1220,5085011,3,0,animal


In [103]:
(empty_correct_count + animal_correct_count)/ effnet_stage_1_pivot.shape[0]

0.92336217552534

In [ ]:
#Predicted animal events:
    #Take avg conf score of all animal predictions
#predicted empty events:
    #Take avg conf score of all image

In [75]:


effnet_animal_event_ids = effnet_stage_1_pivot[~(effnet_stage_1_pivot['event_prediction'] == 'empty')]['event_id']
effnet_stage_1_animal = effnet_stage_1[effnet_stage_1['event_id'].isin(effnet_animal_event_ids)]
effnet_stage_1_animal[effnet_stage_1_animal['class_name'] == 'animal']
effnet_stage_1_animal_conf = effnet_stage_1_animal[['event_id','conf']].groupby('event_id').mean()

effnet_stage_1_blank = effnet_stage_1[~effnet_stage_1['event_id'].isin(effnet_animal_event_ids)]
effnet_stage_1_blank_conf = effnet_stage_1_blank[['event_id','conf']].groupby('event_id').mean()

effnet_conf_scores = pd.concat([effnet_stage_1_animal_conf, effnet_stage_1_blank_conf])

In [82]:
effnet_conf_scores = pd.concat([effnet_stage_1_animal_conf, effnet_stage_1_blank_conf])

,conf
event_id,
2009543,0.751473
2009651,0.587925
2009968,0.589413
2010190,0.977780
2011922,0.568117
...,...
SSWI000000020365850,0.700973
SSWI000000020365917,0.819462
SSWI000000020491540,0.643885


# Phase 2- Species

## Yolo

In [134]:
#Read lines from txt results file

toppath = 'phase 2-species/'
txt_file = 'output_no_wolf.txt'

with open(toppath + txt_file, 'r') as f:
    original_lines = f.readlines()

In [8]:
#Txt file to dataframe
#We are taking the TOP confidence score species for each image as the species
#Count is the count for all bounding boxes drawn in image

rows = []

for item in original_lines:
    
    #Class number
    item_split_1 = item.split(' Bbox[list]')[0]
    class_conf_list = item_split_1.split(';')[1:-2]
    
    conf_score_max = 0
    
    i = 0    
    for class_conf in class_conf_list:
        conf_score_int = float(class_conf.split(',Conf:')[1])
        
        
        if conf_score_int > conf_score_max:
            conf_score_max = conf_score_int
            max_index = i
            
        i += 1
    try:
        class_string = class_conf_list[max_index].split(',')[0]
        class_num = float(class_string.split(':')[1])
        class_num = int(class_num)
    except:
        class_num = 'Blank'

    #Counts
    count_split_1 = item.split(' Bbox[list]')[-1]
    count_split_2 = count_split_1.replace(';', ',')  
    count_split_3 = count_split_2.split(',')
    count = int((len(count_split_3)-1)/4)
        
    #Filename
    file_split_1 = item.split('Filename: ')[1]
    file_name = file_split_1.split(';')[0].split('.jpg')[0]
        
    row = [file_name, class_num, count, conf_score_max]
    
    rows.append(row)       

predictions = pd.DataFrame(rows, columns = ['filename', 'class', 'count', 'conf_score_max'])

Species
1. Events with multiple images of same class will be labeled the majority class
- For event to be labeled Blank, all images must be blank
2. Events with all different labels will get labelled with highest confidence score 

Counts
1. Events with multiple images of same class- we will take the max of the majority class
2. Events with all different labels will get labelled with the count of highest conf score label


In [10]:
#Event column- remove A,B,C appendix
predictions['event_id'] = predictions['filename'].str[:-1]


## Events with muliple images of same class
#Species determination
#Groupby event, class
event_class_group_count = pd.DataFrame(predictions.groupby(['event_id', 'class']).count()['filename']).reset_index().rename(columns = {'filename': 'count_rows'})

#Filter by rows that have count greater than 1
event_class_group_count_majority = event_class_group_count[event_class_group_count['count_rows'] > 1]

#Remove blank events that are not a consensus for all 3 images
event_class_group_count_majority = event_class_group_count_majority[~((event_class_group_count_majority['class'] == 'Blank') &
                                (event_class_group_count_majority['count_rows'] < 3))]

#Count determination
event_class_group_majority_counts = pd.merge(predictions, event_class_group_count_majority,
         how = 'inner',
         on = ['event_id', 'class'])

event_class_majority_counts = event_class_group_majority_counts[['event_id', 'count']].groupby(['event_id']).max().reset_index()

event_max_conf_score = event_class_group_majority_counts[['event_id', 'conf_score_max']].groupby(['event_id']).max().reset_index()

majority_df = pd.merge(event_class_majority_counts, event_class_group_count_majority,
         on = 'event_id',
         how = 'left')[['event_id', 'count','class']]

majority_df = pd.merge(majority_df, event_max_conf_score,
         on = 'event_id',
         how = 'left')

## Events with all different labels
predictions_single_count = predictions[~predictions['event_id'].isin(majority_df['event_id'])]

event_list = []
pred_list = []
count_list = []
conf_score_max_list = []

previous_event = ''
conf_score_group_max = 0
pred_class = ''
conf_score_group_max = 0

predictions_single_count_grouped = predictions_single_count.groupby(['event_id', 'class',
                                                                     'conf_score_max'])

for group_name, group in predictions_single_count_grouped:
        
    current_event = group_name[0]
    current_class = group['class'].iloc[0]
    current_conf_score = group['conf_score_max'].iloc[0]
    current_count = group['count'].iloc[0]
    
    #Check if we are looking at a new event
    if current_event != previous_event:
        conf_score_max_list.append(conf_score_group_max)
        conf_score_group_max = 0
        event_list.append(previous_event)
        pred_list.append(pred_class)
        count_list.append(current_count)
        
    if conf_score_group_max < current_conf_score:
        pred_class = current_class
        conf_score_group_max = current_conf_score
        
    previous_event = current_event 

conf_score_max_list.append(conf_score_group_max)
event_list.append(previous_event)
pred_list.append(pred_class)
count_list.append(current_count)
    
single_class_df = pd.DataFrame(list(zip(event_list,
              pred_list,
              count_list,
              conf_score_max_list)),
            columns = ['event_id', 'class', 'count','conf_score_max'])

predictions_by_events_df = pd.concat([majority_df, single_class_df])


yolo_predictions_by_events_df = pd.merge(predictions_by_events_df, labels,
         how = 'left',
         on = 'class')

truth_pred_yolo_df = pd.merge(truth_file_no_other, yolo_predictions_by_events_df,
                         how = 'left',
                         left_on = 'event_id',
                         right_on = 'event_id')
yolo_preds_df = truth_pred_yolo_df[['event_id', 'class', 'conf_score_max', 'count', 'species']]

yolo_preds_df = yolo_preds_df.rename(columns = {'class': 'pred_class',
                                               'conf_score_max': 'conf_score',
                                                'count': 'pred_count',
                                               'species': 'pred_class_name'})




## EffNet, Megadetecter

In [11]:
model_results = pd.read_json('phase 2-species/phase2_efficientnetb5_classifications.json')
df = model_results['phase2_classification_results'].apply(pd.Series)
df['event_id'] = df['id'].str.split('.jpg').str[0]
df['event_id'] = df['event_id'].str[:-1]

In [12]:
## for index, row in df.iloc[:5].iterrows():
preds_dict = {}
for index, row in df.iterrows():
  event_id = str(row['event_id'])
  pred_class = row['class']
  pred_conf = row['conf']
  #print(event_id, pred_class, pred_conf)
  
  result_dict = {
        "class": pred_class,
        "conf": pred_conf
    }
  
  if event_id in preds_dict:
    preds_dict[event_id].append(result_dict)
  else:
    preds_dict[event_id] = [result_dict]

In [13]:

final_preds_dict = {}
for key, value in preds_dict.items():
  event_id = key
  counts = Counter(d['class'] for d in value)

  ## if all 3 predictions are different, defer to class with highest confidence
  if len(counts) == 3:
    highest_conf = max([x['conf'] for x in value])
    pred_class = [x['class'] for x in value if x['conf']==highest_conf][0]
  
  ## if there is an even number of predictions (2), defer to class with higher confidence
  elif sum(counts.values()) < 3:
    highest_conf = max([x['conf'] for x in value])
    pred_class = [x['class'] for x in value if x['conf']==highest_conf][0]

  ## otherwise, class is based on majority class, conf score is based on highest score for
  ## majority class
  else:
    most_common = {'most_common': counts.most_common(1)[0][0]}
    pred_class = most_common['most_common']
    highest_conf = max([x['conf'] for x in value if x['class'] == most_common['most_common']])

  final_preds_dict[event_id] = [pred_class, highest_conf]

In [14]:
eff_net_preds_df = pd.DataFrame.from_dict(final_preds_dict,  orient='index').reset_index()
eff_net_preds_df.columns=['event_id', 'pred_class', 'conf_score']
label_mapping = dict({0:'bear', 1:'cottontail_snowshoehare', 2:'coyote', 3:'deer', 4:'elk', 5:'foxgray_foxred', 6:'opossum', 7:'raccoon', 8:'turkey', 9:'wolf'})
eff_net_preds_df['pred_class_name'] = eff_net_preds_df['pred_class'].map(label_mapping)


## Consolidate Predictions

In [15]:
eff_net_preds_df['model'] = 'eff_net'
yolo_preds_df['model'] = 'yolo'

In [16]:
ensemble_pred_df = pd.concat([eff_net_preds_df, yolo_preds_df])

In [17]:
ensemble_max_conf_lookup = ensemble_pred_df[['event_id', 'conf_score']].groupby(['event_id']).max().reset_index()


In [18]:
ensemble_max_conf_pred_df =pd.merge(ensemble_pred_df,ensemble_max_conf_lookup,
         how = 'inner',
         on = ['event_id', 'conf_score'])

In [19]:
ensemble_result_metrics = pd.merge(truth_file_no_other, ensemble_max_conf_pred_df,
         how = 'left',
         on = ['event_id'])

In [20]:
y_true = ensemble_result_metrics['true_class_name']
y_pred = ensemble_result_metrics['pred_class_name']
label_list = np.unique(y_true)

target_names = label_list
print(classification_report(y_true, y_pred, target_names=target_names))


                         precision    recall  f1-score   support

                   bear       0.68      0.81      0.74       130
cottontail_snowshoehare       0.62      0.37      0.46       121
                 coyote       0.47      0.33      0.39       135
                   deer       0.82      0.34      0.48       191
                    elk       0.86      0.48      0.61       123
         foxgray_foxred       0.36      0.39      0.37        70
                opossum       0.83      0.38      0.53        52
                raccoon       0.67      0.41      0.51       119
                 turkey       0.47      0.69      0.56        64
                   wolf       0.28      0.87      0.42       126

               accuracy                           0.50      1131
              macro avg       0.61      0.51      0.51      1131
           weighted avg       0.62      0.50      0.51      1131



In [21]:
for model in ['eff_net', 'yolo']:
    ensemble_result_metrics_model = ensemble_result_metrics[ensemble_result_metrics['model'] == model]
    
    y_true = ensemble_result_metrics_model['true_class_name']
    y_pred = ensemble_result_metrics_model['pred_class_name']
    label_list = np.unique(y_true)

    target_names = label_list
    print(classification_report(y_true, y_pred, target_names=target_names))

                         precision    recall  f1-score   support

                   bear       0.78      0.82      0.80       109
cottontail_snowshoehare       0.42      0.23      0.30        83
                 coyote       0.29      0.11      0.16        84
                   deer       0.82      0.21      0.33       136
                    elk       0.94      0.46      0.62        96
         foxgray_foxred       0.22      0.21      0.21        43
                opossum       0.76      0.33      0.46        39
                raccoon       0.56      0.27      0.37        73
                 turkey       0.49      0.67      0.56        55
                   wolf       0.28      0.95      0.43       116

               accuracy                           0.45       834
              macro avg       0.56      0.43      0.42       834
           weighted avg       0.58      0.45      0.44       834

                         precision    recall  f1-score   support

                   be

/Users/sleung2/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sleung2/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sleung2/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:

yolo_metrics = pd.merge(yolo_preds_df, truth_file_no_other, 
         how = 'left',
         on = ['event_id'])


In [23]:
yolo_metrics_not_null = yolo_metrics[~yolo_metrics['pred_class_name'].isnull()]
yolo_metrics_not_null = yolo_metrics_not_null[yolo_metrics_not_null['true_class_name'] != 'wolf']


In [24]:

y_true = yolo_metrics_not_null['true_class_name']
y_pred = yolo_metrics_not_null['pred_class_name']
label_list = np.unique(y_true)

target_names = label_list
print(classification_report(y_true, y_pred, target_names=target_names))

    
    

                         precision    recall  f1-score   support

                   bear       0.63      0.92      0.75       127
cottontail_snowshoehare       0.87      0.65      0.75       106
                 coyote       0.56      0.65      0.60       131
                   deer       0.75      0.56      0.64       176
                    elk       0.76      0.50      0.61       115
         foxgray_foxred       0.35      0.51      0.42        69
                opossum       0.82      0.69      0.75        45
                raccoon       0.73      0.66      0.69       106
                 turkey       0.64      0.83      0.72        60

               accuracy                           0.66       935
              macro avg       0.68      0.66      0.66       935
           weighted avg       0.69      0.66      0.66       935



In [100]:
def images_to_events(image_df, image_id_col = 'id'):
    '''Convert images to events'''

    return image_df['id'].apply(lambda x: x[:-5] if 'SSWI' in x else x.split('_')[0])

def group_events(prediction_image_df):
    '''Convert predictions in terms of events'''
    prediction_event_df = pd.pivot_table(prediction_image_df, index = 'event_id',
              columns = 'class_name',
              values = 'class',
              aggfunc=len, fill_value=0).reset_index() #len(x.unique())).fillna(0)
    return prediction_event_df

def interpret_event_predictions(prediction_event_df):
    return prediction_event_df.apply(lambda x : 'empty' if x['animal'] == 0 else 'animal', axis = 1)

def conf_score_calcs(stage_1, stage_1_pivot, model_type):
    animal_event_ids = stage_1_pivot[~(stage_1_pivot['event_prediction'] == 'empty')]['event_id']
    stage_1_animal = stage_1[stage_1['event_id'].isin(animal_event_ids)]
    stage_1_animal[stage_1_animal['class_name'] == 'animal']
    stage_1_animal_conf = stage_1_animal[['event_id','conf']].groupby('event_id').mean()

    stage_1_blank = stage_1[~stage_1['event_id'].isin(animal_event_ids)]
    stage_1_blank_conf = stage_1_blank[['event_id','conf']].groupby('event_id').mean()

    effnet_conf_scores = pd.concat([effnet_stage_1_animal_conf, effnet_stage_1_blank_conf])
    
    return effnet_conf_scores

## YOLO
def yolo_blank_read_file(top_path, yolo_file):
    '''Convert Yolo txt file to predictions df by event'''

    with open(top_path + yolo_file, 'r') as f:
        blank_yolo_lines = f.readlines()

    yolo_image_prediction_list = []
    yolo_blank_prediction_list = []


    for line in blank_yolo_lines:

        image_name = line.split('Filename: ')[1].split('; ')[0]
        prediction = line.split('; ')[1].split('; Bbox[list]')[0]

        yolo_image_prediction_list.append(image_name)
        yolo_blank_prediction_list.append(prediction)

    yolo_stage_1_prediction_df = pd.DataFrame({'id': yolo_image_prediction_list, 'image_prediction': yolo_blank_prediction_list})

    #Create necessary columns
    yolo_stage_1_prediction_df['event_id'] = images_to_events(yolo_stage_1_prediction_df)
    yolo_stage_1_prediction_df['class_name'] = yolo_stage_1_prediction_df['image_prediction'].apply(lambda x: 'empty' if x == '' else 'animal')
    yolo_stage_1_prediction_df['class'] = yolo_stage_1_prediction_df['class_name'].apply(lambda x: 0 if x == 'animal' else 1)

    yolo_stage_1_prediction_grouped = group_events(yolo_stage_1_prediction_df)

    yolo_stage_1_prediction_grouped['event_prediction'] =  interpret_event_predictions(yolo_stage_1_prediction_grouped)

    return yolo_stage_1_prediction_grouped

## Efficientnet
def effnet_blank_read_file(top_path, yolo_file):
    '''Convert Effnet json file to predictions df by event'''

    effnet_stage_1 = pd.read_json(top_path + yolo_file)
    effnet_stage_1 = effnet_stage_1['phase1_classification_results'].apply(pd.Series)

    effnet_stage_1['event_id'] = images_to_events(effnet_stage_1)

    effnet_stage_1_grouped = group_events(effnet_stage_1)

    effnet_stage_1_grouped['event_prediction'] =  interpret_event_predictions(effnet_stage_1_grouped)

    conf_scores = conf_score_calcs(effnet_stage_1, effnet_stage_1_grouped, 'effnet')
    
    effnet_stage_1_pred_conf = pd.merge(effnet_stage_1_grouped, conf_scores,
         how = 'left',
         left_on = 'event_id',
         right_index = True)[['event_id', 'event_prediction', 'conf']]

    return effnet_stage_1_pred_conf

top_path = '/Users/sleung2/Documents/MIDS Program/capstone/ensemble/phase 1-blank/'
yolo_file = 'phase1_yolo.txt'
effnet_file = 'phase1_efficientnetb0_classifications.json'

yolo_stage_1_prediction_grouped = yolo_blank_read_file(top_path, yolo_file)
effnet_stage_1_pred_conf = effnet_blank_read_file(top_path, effnet_file)



In [101]:
common = effnet_stage_1_pred_conf[effnet_stage_1_pred_conf['event_id'].isin(yolo_stage_1_prediction_grouped.event_id)]



In [115]:
common_merged = pd.merge(common, yolo_stage_1_prediction_grouped,
         on = 'event_id',
         how = 'left')

common_merged = common_merged.rename(columns = {'event_prediction_x': 'effnet_pred',
                                               'event_prediction_y': 'yolo_pred'})
common_merged = common_merged.drop(columns = ['animal', 'empty'])

In [116]:
common_merged

,event_id,effnet_pred,conf,yolo_pred
0,SSWI000000002741773,empty,0.661273,empty
1,SSWI000000004032002,animal,0.999361,animal
2,SSWI000000004068822,animal,0.999964,animal
3,SSWI000000004248668,animal,0.656425,animal
4,SSWI000000004277676,animal,0.807575,animal
...,...,...,...,...
1199,SSWI000000023457314,animal,0.880704,animal
1200,SSWI000000023457319,animal,0.995188,animal
1201,SSWI000000023494506,animal,0.889536,animal
1202,SSWI000000023514111,animal,0.989966,animal


In [117]:
common_merged['match'] = common_merged['effnet_pred'] == common_merged['yolo_pred']

In [129]:
def ensemble_pred_func(ensemble_row, conf_thresh):

    if ensemble_row['effnet_pred'] == 'empty' and ensemble_row['conf'] < conf_thresh:
        ensemble_pred =  ensemble_row['yolo_pred']
    else:
        ensemble_pred = ensemble_row['effnet_pred']
        
    return ensemble_pred

common_merged['ensemble_pred'] = common_merged.apply(lambda x: ensemble_pred_func(x, float(.75)), axis = 1)